In [1]:
from padercontrib.database.fearless import Fearless
import padertorch as pt
import paderbox as pb
import numpy as np
import paderbox as pb
from padercontrib.database.iterator import AudioReader
from paderbox.transform import stft,fbank
import scipy
import torch.nn as nn
import pydub
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import pandas as pd
from pydub import AudioSegment
import torch
from functools import partial
import lazy_dataset
from padertorch.data.utils import collate_fn
import json,pickle
from loss_functions2 import AngularPenaltySMLoss2
from torch.nn.utils.rnn import pad_sequence

In [2]:
db = Fearless()
ds = db.get_dataset('Train_segment')
lb = LabelEncoder()
dev = torch.device("cuda")

In [14]:
def prepare_data(example, batch_size):
        
        #db = Fearless()
        ds = example
        #tr_dict = {}
        #tr_dict['features'] = {}
        #tr_dict['label_array'] = {}
        
        def prep_features(example):
            
            padded_audio = []
            fbank_data = []
            #\\\ Obtain audio segments from the dataset\\\
            #\\\ If segments smaller than 4secs, pad with silence. Else, extract 4secs from larger audio segments \\\
            audio_sam = AudioSegment.from_wav(example['audio_path']['observation'])
            if audio_sam.duration_seconds < 4:
                pad_ms = (4 - audio_sam.duration_seconds)*1000
                silence = AudioSegment.silent(duration=pad_ms) # milliseconds of silence needed
                padded = audio_sam + silence
                
            elif audio_sam.duration_seconds >= 4:
                pad_ms = 0
                audio_sam = audio_sam[0:4000]
                silence = AudioSegment.silent(duration=pad_ms)
                padded = audio_sam + silence        
      
            a = padded.get_array_of_samples()
            b = np.array(a)
            padded_audio.append(b)
            
            #\\\ Compute the 64 dimensional filter banks for the 4secs fixed length audio segments\\\
    
            f_banks = fbank(padded_audio, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                            stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                            window=scipy.signal.windows.hamming, denoise=False)
            fbank_data.append(f_banks)
            tens_fbank = torch.FloatTensor(fbank_data)
            tens_fbank = torch.squeeze(tens_fbank,0)
            #tr_dict['features'] = tens_fbank.to(dev)
            example['features'] = (tens_fbank)
            
            return example
        
        def prep_label(example):
            #label_dict = pb.io.load_json(path = 'labels_argmax.json')
            label_dict2 = pb.io.load_json(path = 'labels_167_hot.json')
            label_dict3 = pb.io.load_json(path = 'labels_167.json')
            #print (example)
        #    if example['speaker_id'] in label_dict.keys():
        #    
        #        pos = label_dict[example['speaker_id']]
        #        example['label_array'] = np.array(pos) 
        #        
            if example['speaker_id'] in label_dict2.keys():
            
                pos = label_dict2[example['speaker_id']]
                example['label_hot'] = np.array(pos)
            
        
            if example['speaker_id'] in label_dict3.keys():
            
                pos = label_dict3[example['speaker_id']]
                example['label_array'] = np.array(pos)
            return example
            
        def array(example):
            audio = pb.io.load_audio(example['audio_path']['observation'])
            example['array_samples'] = audio
            return example
        
        def stack(example):
            example['features'] = np.stack(example['features'])            
            example['label_hot'] = np.stack(example['label_hot'])
            example['label_array'] = np.stack(example['label_array'])
            example['array_samples'] = np.stack(example['array_samples'])
            return example
        
        example = (example
                   .map(prep_features)
                   .map(prep_label)
                   .map(array)
                   .batch(batch_size).map(collate_fn)
                   .map(stack))
                   #.prefetch(buffer_size=8, num_workers=1))                
                
        return example

In [15]:
train_167 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/train_167.json")
train_167_lazy = lazy_dataset.new(train_167['Train_167'])
dev_167 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/dev_167.json")
dev_167_lazy = lazy_dataset.new(dev_167['Dev_167'])

In [16]:
c = prepare_data(dev_167_lazy,1)
c[0]

{'audio_path': {'observation': ['/net/db/fearless/Audio/Segments/ASR_track2/Dev/FS02_ASR_track2_dev_0001.wav']},
 'dataset': ['Dev_segment'],
 'example_id': ['FS02_dev_0001'],
 'num_samples': [37280],
 'speaker_id': ['GNC1'],
 'transcription': ['ROG AND WED LIKE UH ZERO AND FOUR ONES [unk]'],
 'features': array([[[[ 9661.261  , 30697.29   , 11828.361  , ...,   138.81853,
             242.2305 ,   386.81943],
          [ 9402.513  , 30240.041  , 11291.79   , ...,   334.49576,
             230.19894,   180.59735],
          [ 9084.528  , 30423.904  , 11309.22   , ...,   242.9561 ,
             214.24562,   270.18872],
          ...,
          [ 9960.553  , 32644.283  , 14163.003  , ...,   167.7948 ,
             115.86401,   164.84752],
          [10043.176  , 32438.25   , 13636.93   , ...,   196.83505,
             303.5107 ,   283.02075],
          [10434.826  , 19190.426  ,  7654.5444 , ...,   137.92403,
             156.294  ,   211.50085]]]], dtype=float32),
 'label_hot': array([[0.

In [28]:
c[0]['features'].shape

(1, 1, 275, 64)

In [139]:
segmenter = pt.data.segment.Segmenter(length=32000,include_keys='array_samples',padding=True,copy_keys=True,mode='constant')
segmented=[]
for i in c:
    if i['num_samples']>=[32000]:
        segmented.append(segmenter(i))

In [154]:
segmented_2 = []
for i in segmented:
    for j in i:
        segmented_2.append(j)

In [182]:
def prep_example(example):
    audio = pb.io.load_audio(example['audio_path']['observation'])
    print(audio)
    example['array_samples'] = audio
    return example

In [184]:
dev_167_lazy[0]

{'audio_path': {'observation': '/net/db/fearless/Audio/Segments/ASR_track2/Dev/FS02_ASR_track2_dev_0001.wav'},
 'dataset': 'Dev_segment',
 'example_id': 'FS02_dev_0001',
 'num_samples': 37280,
 'speaker_id': 'GNC1',
 'transcription': 'ROG AND WED LIKE UH ZERO AND FOUR ONES [unk]'}

In [183]:
for i in dev_167_lazy:
    prep_example(i)

[-0.00125122 -0.00582886 -0.01693726 ...  0.03555298  0.03326416
  0.03088379]
[ 0.01803589  0.02593994  0.03265381 ... -0.0166626  -0.01528931
 -0.00906372]
[-0.01611328 -0.01150513 -0.01437378 ...  0.02835083  0.02642822
  0.02520752]
[0.01495361 0.01779175 0.01635742 ... 0.01333618 0.00622559 0.00021362]
[-0.03027344 -0.0357666  -0.01751709 ...  0.02655029  0.01773071
  0.00427246]
[ 0.01638794  0.01608276  0.00601196 ... -0.04248047 -0.04507446
 -0.05297852]
[-0.00622559 -0.00839233 -0.01040649 ... -0.00842285 -0.01107788
 -0.00836182]
[-0.00421143 -0.00827026 -0.00216675 ... -0.01278687 -0.01702881
 -0.01681519]
[-0.00564575 -0.00466919 -0.00564575 ... -0.00097656 -0.00332642
 -0.00631714]
[-0.03411865 -0.03225708 -0.02972412 ...  0.04943848  0.05953979
  0.06607056]
[-0.00250244 -0.00613403 -0.0055542  ...  0.0239563   0.02450562
  0.02182007]
[-0.0111084   0.00115967  0.00811768 ...  0.00488281  0.00064087
 -0.00411987]
[ 0.07025146  0.06933594  0.06115723 ... -0.03411865 -0.028

[-0.02593994 -0.02581787 -0.02450562 ... -0.03323364 -0.02200317
 -0.00292969]
[-0.01434326 -0.01705933 -0.0177002  ... -0.00668335 -0.00518799
 -0.00344849]
[0.00466919 0.00671387 0.00704956 ... 0.01321411 0.01235962 0.01528931]
[-0.07733154 -0.06402588 -0.05249023 ...  0.06817627  0.06933594
  0.06976318]
[0.01223755 0.01687622 0.01580811 ... 0.01919556 0.01898193 0.01794434]
[-0.00408936  0.00299072  0.01226807 ...  0.02374268  0.02160645
  0.01916504]
[-0.01358032 -0.01287842 -0.012146   ...  0.00317383  0.00436401
 -0.00842285]
[ 0.00497437  0.00814819  0.01266479 ... -0.00079346 -0.00396729
 -0.0043335 ]
[ 0.01016235  0.00805664 -0.01528931 ... -0.01495361 -0.05789185
 -0.05505371]
[-0.01150513 -0.00830078 -0.00863647 ... -0.00415039 -0.0007019
 -0.00042725]
[ 0.03131104  0.01800537  0.02304077 ... -0.01171875 -0.01370239
 -0.01708984]
[-0.00668335 -0.00421143 -0.00216675 ...  0.00048828 -0.00012207
 -0.00033569]
[0.02798462 0.03530884 0.03161621 ... 0.02990723 0.01528931 0.02810

[ 0.00509644 -0.00845337 -0.01538086 ...  0.02020264  0.02154541
  0.02148438]
[0.0151062  0.0161438  0.01846313 ... 0.02026367 0.01931763 0.02191162]
[0.06231689 0.04605103 0.0484314  ... 0.02261353 0.0173645  0.01141357]
[-0.02062988 -0.02374268 -0.01678467 ...  0.01461792  0.01751709
  0.0161438 ]
[0.02038574 0.01818848 0.01556396 ... 0.01196289 0.01452637 0.01425171]
[-0.00500488 -0.01028442 -0.01489258 ...  0.01953125  0.01885986
  0.02072144]
[ 0.00262451  0.00036621 -0.0022583  ... -0.01968384 -0.0211792
 -0.01956177]
[-0.05639648 -0.06060791 -0.06225586 ...  0.02102661  0.02392578
  0.02523804]
[0.00180054 0.00088501 0.00186157 ... 0.0090332  0.0171814  0.02261353]
[-0.03173828 -0.03463745 -0.03723145 ...  0.00531006 -0.01266479
 -0.02566528]
[-0.04016113 -0.03387451 -0.01977539 ...  0.03189087  0.03356934
  0.03100586]
[-0.00357056 -0.00161743  0.00350952 ...  0.00811768  0.00820923
  0.00601196]
[0.00350952 0.003479   0.01403809 ... 0.04727173 0.0524292  0.04290771]
[-0.03146

[ 0.00234985 -0.00149536 -0.00683594 ... -0.00735474 -0.00863647
 -0.00872803]
[-0.00146484 -0.00317383 -0.00332642 ... -0.02355957 -0.02459717
 -0.05404663]
[-0.00241089 -0.00436401 -0.00558472 ...  0.02468872  0.02905273
  0.02416992]
[-0.00582886 -0.00457764 -0.00314331 ... -0.00570679 -0.00442505
 -0.00592041]
[0.07720947 0.07012939 0.09503174 ... 0.02709961 0.02886963 0.03765869]
[0.01296997 0.00933838 0.01202393 ... 0.01870728 0.0262146  0.03585815]
[-0.00976562  0.00039673  0.01080322 ... -0.02658081 -0.01837158
 -0.00970459]
[ 0.00180054 -0.00363159 -0.00717163 ...  0.00634766  0.0090332
 -0.00091553]
[-0.05822754 -0.03500366  0.00915527 ... -0.01730347 -0.02185059
 -0.02508545]
[-0.00152588 -0.00515747 -0.00299072 ... -0.01278687 -0.01156616
 -0.01171875]
[0.00057983 0.00579834 0.0088501  ... 0.02099609 0.02297974 0.02050781]
[0.0413208  0.04284668 0.03564453 ... 0.03543091 0.02947998 0.01849365]
[ 0.01477051  0.01959229  0.00177002 ... -0.00335693 -0.0065918
 -0.00796509]
[-0

[ 0.01107788  0.00265503 -0.00253296 ...  0.00082397  0.00604248
  0.00463867]
[-0.00698853 -0.00570679  0.00109863 ... -0.00839233 -0.01010132
 -0.01147461]
[-0.01815796 -0.01528931 -0.01449585 ... -0.00192261 -0.00802612
  0.00531006]
[-0.00592041 -0.00576782 -0.00509644 ...  0.0274353   0.02590942
  0.0279541 ]
[-0.01620483 -0.01849365 -0.01715088 ... -0.02389526 -0.02105713
 -0.01803589]
[ 0.02719116  0.03112793  0.02493286 ... -0.05432129 -0.04193115
 -0.00787354]
[-0.00564575 -0.00808716 -0.01049805 ... -0.00527954 -0.00552368
 -0.00784302]
[-0.00769043 -0.01379395 -0.00909424 ... -0.00079346 -0.00378418
 -0.00540161]
[-0.00167847 -0.00302124 -0.00473022 ...  0.01846313  0.01599121
  0.01422119]
[-0.00231934  0.00228882  0.00195312 ... -0.01190186 -0.01019287
 -0.01144409]
[-0.12939453 -0.14358521 -0.13262939 ...  0.01638794  0.01513672
  0.01879883]
[ 0.00918579  0.01080322  0.01245117 ...  0.00024414 -0.00131226
 -0.00259399]
[-0.02160645 -0.02359009 -0.01708984 ...  0.00177002

...

[ 0.00445557 -0.01031494 -0.02520752 ...  0.08428955  0.09020996
  0.08874512]
[ 0.01141357  0.01400757  0.01260376 ... -0.00192261  0.00036621
  0.00198364]
[-0.00634766 -0.00994873  0.01177979 ... -0.0144043  -0.01358032
 -0.00543213]
[-4.50439453e-02 -2.02636719e-02  9.15527344e-05 ...  2.30712891e-02
  1.09863281e-02  3.08227539e-03]
[0.02658081 0.02099609 0.02078247 ... 0.05136108 0.03573608 0.01480103]
[ 0.00915527  0.00234985  0.01141357 ... -0.01382446 -0.01376343
 -0.02139282]
[ 0.02130127  0.02029419  0.02023315 ...  0.00091553 -0.0010376
 -0.00140381]
[ 0.00167847 -0.00112915 -0.00421143 ... -0.00949097 -0.00524902
 -0.0050354 ]
[-0.04650879 -0.03964233 -0.03445435 ... -0.01452637 -0.02151489
 -0.02532959]
[0.01705933 0.02078247 0.02053833 ... 0.02700806 0.02606201 0.03082275]
[-0.0010376  -0.0128479  -0.02288818 ... -0.02532959 -0.02511597
 -0.02206421]
[-0.04214478 -0.03509521 -0.02593994 ... -0.02481079 -0.02609253
 -0.03121948]
[0.02288818 0.03756714 0.04379272 ... 0.003

[ 0.00671387  0.0090332   0.00732422 ... -0.01367188 -0.01760864
 -0.02029419]
[-0.00418091 -0.00479126 -0.0065918  ...  0.00180054 -0.00170898
 -0.00094604]
[ 0.01626587  0.00979614  0.01791382 ... -0.02032471 -0.02371216
 -0.02822876]
[-0.01266479 -0.0083313  -0.00378418 ...  0.00268555  0.00500488
  0.00527954]
[-0.00622559 -0.01596069 -0.01919556 ...  0.02749634  0.02603149
  0.02526855]
[-0.0022583   0.00149536 -0.0022583  ... -0.00158691 -0.01678467
 -0.02297974]
[ 0.00967407  0.01885986  0.0222168  ... -0.21520996 -0.23599243
 -0.2507019 ]
[-0.01266479 -0.01098633 -0.00680542 ...  0.01898193  0.01031494
 -0.00787354]
[-0.00598145 -0.04324341 -0.05780029 ...  0.00646973  0.00894165
 -0.00180054]
[0.01525879 0.02383423 0.02713013 ... 0.01071167 0.01422119 0.02029419]
[-0.0118103  -0.00790405  0.00045776 ...  0.0244751   0.00540161
 -0.00170898]
[-0.00521851 -0.0166626  -0.01480103 ...  0.0262146   0.03933716
  0.02191162]
[-0.00875854 -0.01052856 -0.00714111 ... -0.02429199 -0.026

[-0.02752686 -0.02185059 -0.02331543 ... -0.00405884  0.01077271
  0.02746582]
[ 0.03741455 -0.00549316 -0.03225708 ... -0.01315308 -0.00799561
 -0.00643921]
[-0.02423096 -0.02142334 -0.0213623  ...  0.04418945  0.03979492
  0.03433228]
[-0.0171814  -0.01040649 -0.00912476 ... -0.01745605 -0.01672363
 -0.01831055]
[0.01809692 0.02609253 0.0223999  ... 0.03265381 0.04043579 0.03305054]
[-0.00967407 -0.01074219 -0.01190186 ...  0.00170898  0.00167847
  0.00228882]
[-0.01675415 -0.0145874  -0.01309204 ... -0.02044678 -0.02017212
 -0.02154541]
[ 0.02529907  0.02444458  0.0241394  ... -0.00476074 -0.00262451
 -0.00317383]
[-0.01858521 -0.02633667 -0.0274353  ...  0.01358032  0.01965332
  0.02627563]
[-0.02633667 -0.00082397  0.01831055 ... -0.01339722 -0.01858521
 -0.02078247]
[0.04299927 0.05725098 0.06015015 ... 0.02496338 0.0189209  0.01315308]
[-0.00610352 -0.00384521 -0.00842285 ...  0.00100708  0.00079346
  0.00057983]
[ 0.02023315  0.01174927  0.00985718 ... -0.01486206 -0.01318359
 

[ 0.01608276  0.01959229  0.02044678 ... -0.01953125 -0.019104
 -0.01702881]
[0.03182983 0.02279663 0.0144043  ... 0.01495361 0.01596069 0.01690674]
[0.02041626 0.02203369 0.02035522 ... 0.01422119 0.01364136 0.00973511]
[-0.01525879 -0.00769043  0.00540161 ...  0.00299072  0.00082397
 -0.00158691]
[-0.02487183 -0.00787354 -0.01168823 ...  0.03518677  0.03469849
  0.02392578]
[ 0.00253296  0.00219727  0.00390625 ... -0.00091553  0.00146484
  0.00137329]
[0.0039978  0.0072937  0.00350952 ... 0.00280762 0.00057983 0.00268555]
[-0.01525879 -0.01370239 -0.019104   ...  0.03277588  0.03405762
  0.02593994]
[-0.03128052 -0.02441406 -0.02334595 ... -0.02886963 -0.03341675
 -0.02938843]
[ 0.00302124  0.00689697 -0.00305176 ... -0.01834106 -0.02297974
 -0.0213623 ]
[-0.00164795 -0.00363159 -0.00686646 ...  0.006073    0.00439453
  0.00262451]
[-0.01144409 -0.01321411 -0.01385498 ...  0.00634766  0.00662231
  0.00604248]
[-0.01544189 -0.00918579 -0.00354004 ... -0.00720215 -0.00552368
 -0.007232

KeyboardInterrupt: 

In [69]:
def prepare_data_2(example, batch_size, shuffle = False):
    
    ds = example
 
    def prep_features(example):
        
        
        fbank_data = []
        """ Obtain audio segments from the dataset"""
 
        """ Compute the 64 dimensional filter banks for the 4secs fixed length audio segments"""
        audio = pb.io.load_audio(example['audio_path']['observation'],dtype=np.int16)
        f_banks = fbank(audio, sample_rate=8000, window_length=400, stft_shift=200, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming,denoise=False)
        fbank_data.append(f_banks)
       # print(fbank_data[0].shape)
        float_fbank = np.float32(fbank_data)
        float_fbank = np.squeeze(float_fbank,0)
        
        example['features'] = torch.from_numpy(float_fbank)
        #example['pad_dim'] = example['features'].shape[-2]
        return example
    
    def prep_label(example):
        label_dict = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_167.json')
        label_dict2 = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_167_hot.json')
        
        if example['speaker_id'] in label_dict.keys():
        
            pos = label_dict[example['speaker_id']]
            example['label_array'] = np.array(pos) 
            
        if example['speaker_id'] in label_dict2.keys():
        
            pos = label_dict2[example['speaker_id']]
            example['label_hot'] = np.array(pos)
        return example   
     
    def stack(example):
        
        example['features'] = pad_sequence(example['features'], batch_first=True)
        #pb.visualization.plot.spectrogram(example['features'])
        example['features'] =  torch.unsqueeze(example['features'],1)
        #example['features'] = np.stack(example['features'])
        example['label_array'] = np.stack(example['label_array'])
        example['label_hot'] = np.stack(example['label_hot'])
        
        return example
    
    example = example.map(prep_features)
    example = example.map(prep_label)
    
    example = example.batch(batch_size).map(collate_fn)
     
    example = example.map(stack)
    if shuffle:
        example = example.shuffle()  
    #example = example.prefetch(buffer_size=8, num_workers=8)
            
            
    return example

In [70]:
a = prepare_data_2(train_167_lazy,1)

In [71]:
a[-1],a[0]

({'audio_path': {'observation': ['/net/db/fearless/Audio/Segments/ASR_track2/Train/FS02_ASR_track2_train_35473.wav']},
  'dataset': ['Train_segment'],
  'example_id': ['FS02_train_35473'],
  'num_samples': [7200],
  'speaker_id': ['CAPCOM1'],
  'transcription': ['[unk] OVER'],
  'features': tensor([[[[  602.5853, 25337.2969, 50247.8359,  ...,  1113.2666,
               741.6537,   164.3370],
            [  474.8964, 25724.3516, 51282.1367,  ...,   663.7629,
              1397.6755,   252.0795],
            [  466.1526, 25612.9629, 50960.7578,  ...,   960.7065,
              2037.6437,   644.3087],
            ...,
            [  404.3279, 22413.9258, 40713.5312,  ...,   697.1683,
               574.1517,   169.6190],
            [  324.8476, 20890.8633, 41278.6289,  ...,  1791.0287,
              1844.4097,   188.7019],
            [  417.5273, 20930.2598, 41585.3984,  ...,  1008.0200,
               617.2301,   110.7129]]]]),
  'label_array': array([15]),
  'label_hot': array([[0., 0.

In [10]:
from sklearn.metrics import confusion_matrix
from scipy.special import softmax

class PT_ConvNet(pt.base.Model):
    
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU())
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(32))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=3, stride=2))
            
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2))
        #self.lstm = nn.LSTM(input_size=256, hidden_size=64, num_layers=2, dropout=0.5, bidirectional=True)
        
        self.drop_out = nn.Dropout(0.5)        
        self.fc1 = nn.Linear(256, 128)                
        self.fc2 = nn.Linear(128,167)

    def forward(self, data):

        x = self.layer1(data)
        x = self.layer2(x)        
        x = self.layer3(x)        
        x = self.layer4(x)  
        x = self.layer5(x)
        print(x.shape)
        x = torch.mean(x,3)
        #x,_ = torch.max(x,3)
        print(x.shape)
        x = torch.mean(x,2)
        #x,_ = torch.max(x,2)
        print(x.shape)        
        
        x = x.reshape(x.size(0), -1)
          
        x = self.fc1(x)
        x = self.drop_out(x)
        x = self.fc2(x)
    
        return x
    
    def review():
        pass

In [95]:
import torch
from einops import rearrange

In [132]:
class RNN(pt.base.Model):
    ce_loss = nn.CrossEntropyLoss(reduction='none')
        
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1,64,kernel_size=5,stride=2)
        self.avg2d = nn.AvgPool2d(kernel_size=2,stride=2)
        self.rnn = nn.RNN(64,1024,num_layers=3,batch_first=True)
     
        self.linear1 = nn.Linear(1024,512)
        self.batchnorm = nn.BatchNorm2d(512)
        self.linear2 = nn.Linear(512,167)
        
    def forward(self, data):
        x = data['features']
        x = self.conv(x)
        x = self.avg2d(x)
        x = torch.mean(x,3)
        #print(x.shape)
        x = torch.transpose(x,1,2)
        #print(x.shape)
        x,_ = self.rnn(x)
   
        x = torch.mean(x,1)

        #print(x)
        x = self.linear1(x)
       
        x = self.linear2(x)
        
        return dict(prediction=x)
        
    def review(self, data, outputs):
        
        labels = data['label_array']
        pred = outputs['prediction']
        
        ce = self.ce_loss(pred,labels)
        review = dict(
            loss = ce.mean(),
            buffers = dict(
                labels = labels.data.cpu().numpy(),
                #predictions = asm[0].data.cpu().numpy(),
                predictions = pred.data.cpu().numpy(),
            )  
        )
        return review
    
    def modify_summary(self,summary):
        
        if 'labels' in summary['buffers']:
            labels = np.concatenate(summary['buffers'].pop('labels'))
            predictions = np.concatenate(summary['buffers'].pop('predictions'))
            #prediction_soft = softmax(predictions)
            prediction_max = np.argmax(predictions, axis=-1)
            summary['scalars']['accuracy'] = (prediction_max == labels).mean()
            summary['scalars']['top_k'] = tf.keras.metrics.sparse_top_k_categorical_accuracy(labels,predictions, k=5)
        summary = super().modify_summary(summary)
        
        return summary

In [138]:
model = RNN()

model

RNN(
  size=ModelParameterSize(total_count=5927719, trainable_count=5927719, total_bytes=23710876, trainable_bytes=23710876)
  (conv): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2))
  (avg2d): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (rnn): RNN(64, 1024, num_layers=3, batch_first=True)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=167, bias=True)
)

In [117]:
import sklearn
from sklearn import metrics

    
trainer = pt.train.trainer.InteractiveTrainer(
                model.to('cuda'),
                optimizer=pt.train.optimizer.SGD(lr=0.001),#,nesterov=True,momentum=0.9),
            
                stop_trigger=(100,'iteration'),
                summary_trigger=(10,'iteration'),
            
                )
#trainer.register_hook(pt.train.hooks.LRSchedulerHook(
#torch.optim.lr_scheduler.ExponentialLR(trainer.optimizer.optimizer, gamma=0.1))) #, milestones=[25,40]))
d = trainer.train(c)


  1%|          | 1/100 [00:00<?, ?it/s]

Move trainer, model and optimizer to cuda:0.
10, training/loss: 3.620213973522186
10, training/grad_norm: 182.23126678466798
10, training/lr/param_group_0: 0.0010000000000000002
10, training/accuracy: 0.3
10, training/top_k: 0.6000000238418579
10, training/top_2: 0.5
20, training/loss: 5.499240601062775
20, training/grad_norm: 164.1330123901367
20, training/lr/param_group_0: 0.0010000000000000002
20, training/accuracy: 0.1
20, training/top_k: 0.10000000149011612
20, training/top_2: 0.10000000149011612
30, training/loss: 5.915377521514893
30, training/grad_norm: 149.9888427734375
30, training/lr/param_group_0: 0.0010000000000000002
30, training/accuracy: 0.0
30, training/top_k: 0.0
30, training/top_2: 0.0
40, training/loss: 4.781038451194763
40, training/grad_norm: 128.45771865844728
40, training/lr/param_group_0: 0.0010000000000000002
40, training/accuracy: 0.0
40, training/top_k: 0.10000000149011612
40, training/top_2: 0.10000000149011612
50, training/loss: 5.317311716079712
50, train


100%|██████████| 100/100 [00:08<00:00, 11.64it/s]

100, training/loss: 4.465414190292359
100, training/grad_norm: 106.20809326171874
100, training/lr/param_group_0: 0.0010000000000000002
100, training/accuracy: 0.1
100, training/top_k: 0.30000001192092896
100, training/top_2: 0.20000000298023224
Training ended after 0 epochs and 100 iterations


In [114]:
model = ResNet50()
model

ResNet(
  size=ModelParameterSize(total_count=23818650, trainable_count=23818650, total_bytes=95274600, trainable_bytes=95274600)
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsampl

In [113]:
import torch
import torch.nn as  nn
import torch.nn.functional as F


class Bottleneck(pt.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(pt.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        identity = x.clone()
  
        x = self.relu(self.batch_norm2(self.conv1(x)))
        x = self.batch_norm2(self.conv2(x))
  
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        print(x.shape)
        print(identity.shape)
        x += identity
        x = self.relu(x)
        return x

        
        
class ResNet(pt.Model):
    loss_fn = nn.CrossEntropyLoss(reduction='none')
    def __init__(self, ResBlock, layer_list, num_classes=218, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(512*ResBlock.expansion, 128)
        self.drop_out = nn.Dropout(0.5) 
        self.fc2 = nn.Linear(128,num_classes)
        
    def forward(self, data):
        x = data['features']
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        #x = torch.mean(x,(2,3))
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.drop_out(x)
        x = self.fc2(x)
        
        return dict(prediction=x)
        
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

         
    def review(self, data, outputs):
        
        labels = data['label_array'].to('cuda')
        pred = outputs['prediction'].to('cuda')
        #asm = self.adms_loss(pred, labels)
        #print(asm)
        ce = self.loss_fn(pred,labels)
        review = dict(
            # loss = asm[1].mean(),
            loss = ce.mean(),
            buffers = dict(
                labels = labels.data.cpu().numpy(),
                #predictions = asm[0].data.cpu().numpy(),
                predictions = pred.data.cpu().numpy(),
            )  
        )
        return review
    
    def modify_summary(self,summary):
        
        if 'labels' in summary['buffers']:
            labels = np.concatenate(summary['buffers'].pop('labels'))
            predictions = np.concatenate(summary['buffers'].pop('predictions'))
            #prediction_soft = softmax(predictions)
            prediction_max = np.argmax(predictions, axis=-1)
            summary['scalars']['accuracy'] = (prediction_max == labels).mean()
            summary['scalars']['top_k'] = tf.keras.metrics.sparse_top_k_categorical_accuracy(labels,predictions, k=5)
            summary['scalars']['top_2'] = tf.keras.metrics.sparse_top_k_categorical_accuracy(labels,predictions, k=2)
        summary = super().modify_summary(summary)
        
        return summary
    

        
def ResNet50(num_classes=218, channels=1):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [108]:
del model

In [16]:
trainer.register_validation_hook()

TypeError: register_validation_hook() missing 1 required positional argument: 'validation_iterator'

In [51]:
class Block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super().__init__()
        self.expansion = 1 # No expansion between ResNet blocks
        
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=1
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        
        self.conv2 = nn.Conv2d(
            intermediate_channels, intermediate_channels*4, kernel_size=3, stride=stride, padding=1,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels*4)
        self.conv3 = nn.Conv2d(
            intermediate_channels*4, intermediate_channels*4, kernel_size=1, stride=stride, padding=1,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels*4)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet(pt.Model):
    loss_func = nn.CrossEntropyLoss(reduction='None')
    block = Block
    def __init__(self, layers=[3,4,6,3], image_channels=1, num_classes=167):
        super(ResNet, self).__init__()
        self.in_channels = 32
        self.conv1 = nn.Conv2d(image_channels, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool2d = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        #self.lstm = nn.LSTM(input_size=128, hidden_size=64, num_layers=2, dropout=0.5, bidirectional=True)

        self.layer1 = self._make_layer(
            self.block, layers[0], intermediate_channels=32, stride=1
        )
        self.layer2 = self._make_layer(
            self.block, layers[1], intermediate_channels=64, stride=1
        )
        self.layer3 = self._make_layer(
            self.block, layers[2], intermediate_channels=128, stride=1
        )
        self.layer4 = self._make_layer(
            self.block, layers[3], intermediate_channels=256, stride=1
        )

        self.drop_out = nn.Dropout(0.5)        
        self.fc1 = nn.Linear(256, 128)                
        self.fc2 = nn.Linear(128,167)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        #x = self.maxpool(x)
        x = self.avgpool2d(x)
        x = self.layer3(x)
        #x = self.maxpool(x)
        x = self.avgpool2d(x)
        x = self.layer4(x)
        #x = self.maxpool(x)
        x = self.avgpool2d(x)
        x = torch.mean(x, (2,3))
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.drop_out(x)
        x = self.fc2(x)
        return x
   

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 1:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 1,
                    kernel_size=1,
                    stride=stride,
                ),
                nn.BatchNorm2d(intermediate_channels * 1),
            )

        layers.append(
            self.block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        self.in_channels = intermediate_channels * 1

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)
    
     
    def review():
        pass

In [19]:
class ConvAngularPen(pt.base.Model):
    
    
    def __init__(self, num_classes=167):
        super(ConvAngularPen, self).__init__()
        #self.convlayers = PT_ConvNet()
        #self.adms_loss = AngularPenaltySMLoss2(100, num_classes)
        self.loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
        self.resnet = ResNet()
        
    def forward(self, x, labels=None, embed=False):
        #x = self.convlayers(x['features'])
        x = self.resnet(x['features'])
        if embed:
            return x
        return dict(prediction=x)
    
    def review(self, data, outputs):
        
        labels = data['label_array'].to('cuda')
        pred = outputs['prediction'].to('cuda')
        #asm = self.adms_loss(pred, labels)
        #print(asm)
        ce = self.loss_fn(pred,labels)
        review = dict(
            # loss = asm[1].mean(),
            loss = ce.mean(),
            buffers = dict(
                labels = labels.data.cpu().numpy(),
                #predictions = asm[0].data.cpu().numpy(),
                predictions = pred.data.cpu().numpy(),
            )  
        )
        return review
    
    def modify_summary(self,summary):
        
        if 'labels' in summary['buffers']:
            labels = np.concatenate(summary['buffers'].pop('labels'))
            predictions = np.concatenate(summary['buffers'].pop('predictions'))
            #prediction_soft = softmax(predictions)
            prediction_max = np.argmax(predictions, axis=-1)
            summary['scalars']['accuracy'] = (prediction_max == labels).mean()
            summary['scalars']['top_k'] = tf.keras.metrics.sparse_top_k_categorical_accuracy(labels,predictions, k=5)
        summary = super().modify_summary(summary)
        
        return summary
    


In [ ]:
old_167 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/train_167.json")
dev_167 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/dev_167.json")

In [ ]:
(len(old_167['Train_167']) - len(new_lazy))/len(old_167['Train_167']) * 100 # % data lost from train_segment


In [ ]:
(len(oldev_167['Dev_167']) - len(dev_lazy))/len(oldev_167['Dev_167']) * 100 # % data lost from dev_segment

In [ ]:
labels_167 = pb.io.load_json(path='/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_167.json')
labels_167

In [ ]:
train_sid = db.get_dataset('Train_SID')
dev_sid = db.get_dataset('Dev_SID')
len(train_sid),len(dev_sid)

In [ ]:
sid_train_167 = []
sid_dev_167 = []
for i in train_sid:
    if i['speaker_id'] in labels_167.keys():
        sid_train_167.append(i)
        
for i in dev_sid:
    if i['speaker_id'] in labels_167.keys():
        sid_dev_167.append(i)       

In [ ]:
len(sid_train_167), len(sid_dev_167)

In [ ]:
old_list = old_167['Train_167']
old_list

In [ ]:
new

In [ ]:
j = []
for i in old_list:
    j.append(i['speaker_id'])
    
#m = []
#for i in dev_lazy:
#    m.append(i['speaker_id'])
#    

In [ ]:
unique, counts = np.unique(j, return_counts=True)
dict1 = dict(zip(unique, counts))
len(dict1)

In [ ]:
dict_rev = dict(sorted(dict1.items(), key=lambda item: item[1], reverse=True))
sum(dict_rev.values())

In [ ]:
top100 = {k: dict_rev[k] for k in list(dict_rev)[:100]}
top50 = {k: dict_rev[k] for k in list(dict_rev)[:50]}

spk100 = top100.keys()
spk50 = top50.keys()
val100 = top100.values()
val50 = top50.values()

In [ ]:
train_50 = []
for i in spk50:
    for j in old_list:
        if i==j['speaker_id']:
            train_50.append(j)

In [ ]:
#devlist_100 = sorted(dev_100, key=lambda k: k['audio_path']['observation'])
trainlist_50 = sorted(train_50, key=lambda k: k['audio_path']['observation'])
#len(devlist_100)
len(trainlist_50)

In [ ]:
train_50 = {'Train_50':trainlist_50}
#type(dev_50)

pb.io.dump_json(train_50,'train_50.json')

In [ ]:
labels_167 = pb.io.load_json("labels_167.json")

In [ ]:
m =  (list(spk100))
#m = list(labels_167)
m.sort()
labels = np.array(m)
labels = to_categorical(lb.fit_transform(m))

#labels = np.argmax(labels,axis=0)
new_dict = dict(zip(m,labels))    
new_dict
pb.io.dump_json(new_dict, path = "labels_100_hot.json")

In [ ]:
list(m)

In [ ]:
model1 = ResNet()

model1

In [ ]:
import pb_sed.evaluation.instance_based as sed
from scipy.spatial.distance import euclidean as euc

In [ ]:
arr = [1,2,3,4,5]
arrmax = np.max(softmax(arr))
new_thr = []
thr = [0.1,0.1,1,1]
for i in thr:
    dis = euc(arrmax,i)
    new_thr.append(dis)
   # if arrmax<i:
   #     new_thr.append(0)
   # else:
   #     new_thr.append(arrmax-i)
print(new_thr)

In [ ]:
arr_s = softmax(arr)
np.max(arr_s)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
res = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/eval/result_newce2.json")
y = range(0,167)

In [ ]:
plt.bar(y,res['Thresholds'],width=5.0,color='green')
plt.xlabel('Speakers --> (0-167)'),plt.ylabel('Threshold values')
plt.show()

In [ ]:
from pathlib import Path
import numpy as np
import torch
import padertorch as pt
import padercontrib as pc
import paderbox as pb
from padertorch import Model
from sacred import Experiment, commands
from sklearn import metrics
from tqdm import tqdm
from paderbox.io.new_subdir import get_new_subdir
from paderbox.io import load_json, dump_json
from pprint import pprint
from fearless.sid.data import prepare_data
from scipy.special import softmax
from scipy.spatial.distance import euclidean as euc
from statistics import mean
import tensorflow as tf
import pb_sed.evaluation.instance_based as sed
import lazy_dataset

In [ ]:
dev = pb.io.load_json(path='dev_167_batch.json')

In [ ]:
dev_167_lazy = lazy_dataset.new(dev['Dev_167'])
validation_data = prepare_data(dev_167_lazy,batch_size=1)

In [ ]:
dev_167_lazy

In [ ]:
from pydub import AudioSegment

In [ ]:
audio_sam = AudioSegment.from_wav(dev_167_lazy[0]['audio_path']['observation'])
audio = pb.io.load_audio(dev_167_lazy[0]['audio_path']['observation'],dtype=np.int16)

In [ ]:
max(np.array(audio_sam.get_array_of_samples()))

In [ ]:
f_banks = fbank(audio_sam.get_array_of_samples(), sample_rate=8000, window_length=400, stft_shift=200, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming,denoise=False)

f_banks

In [ ]:
np.float32(audio)

In [ ]:
f_banks = fbank(audio, sample_rate=8000, window_length=400, stft_shift=200, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming,denoise=False)

f_banks

In [ ]:
from fearless.sid.model_resnet_2 import ResNet
from fearless.sid.model_asm import ConvAngularPen

In [ ]:
model_res = ConvAngularPen()

model_res

In [ ]:
exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-03-10-24-31'
ckpt_name = 'ckpt_best_loss.pth'
device = 0
model_SID = Model.from_storage_dir(
    exp_dir, consider_mpi=True, checkpoint_name=ckpt_name
)
model_SID.to(device)
model_SID.eval()

In [ ]:
#Evaluate

with torch.no_grad():
        metric = {'Accuracy':[],'Top_5_Accuracy':[]}
        Accuracy = []
        Precision = []
        Recall = []
        F1 = []
        Top_5 = []
        scores = []
        targets = []
        for example in tqdm(validation_data):
            example = model_SID.example_to_device(example, device)
            output = model_SID(example)
            pred = output['prediction'].cpu().detach().numpy()
            prediction_soft = softmax(pred)
            prediction_max = np.argmax(pred, axis=-1)
            target = example['label_array'].cpu().detach().numpy()
            target_hot = example['label_hot'].cpu().detach().numpy()
            #print(target_hot)
            accuracy = (prediction_max == target).mean()            
           
            Accuracy.append(accuracy)  
            top_5 = tf.keras.metrics.sparse_top_k_categorical_accuracy(target,prediction_soft, k=5)
            top_5 = np.array(top_5).mean()
            Top_5.append(top_5)
            
            precision = metrics.precision_score(target,prediction_max, average='macro', zero_division=0)
            Precision.append(precision)
            recall = metrics.recall_score(target,prediction_max, average='macro', zero_division=0)
            Recall.append(recall)
            f1 = metrics.f1_score(target,prediction_max, average='macro', zero_division=0)
            F1.append(f1)
            
            targets.append(target_hot)            
            scores.append(prediction_soft)
            
        targets_con = np.concatenate((targets))
        scores_con = np.concatenate((scores))
        thr,met = sed.get_optimal_thresholds(targets_con,scores_con,metric='f1')       
        #decisions = scores > thr
        #print(decisions)
        #f1, p, r = sed.fscore(targets, decisions, event_wise=False)
        
        auc = metrics.roc_auc_score(targets_con, scores_con, None)
        
            
        metric['Accuracy'] = np.mean(Accuracy)
        metric['Top_5_Accuracy'] = np.mean(Top_5)
        metric['Precision'] = np.mean(Precision)
        metric['Recall'] = np.mean(Recall)
        metric['F1_score'] = np.mean(F1)
        metric['AUC'] = np.mean(auc)
        metric['Thresholds'] = thr

In [ ]:
targets_con.shape, scores_con.shape

In [ ]:
print(metrics.f1_score(targets_argmax,scores_argmax,average='weighted'), ':', 'weighted')
print(metrics.f1_score(targets_argmax,scores_argmax,average='macro'), ':', 'macro')
print(metrics.f1_score(targets_argmax,scores_argmax,average='micro'), ':', 'micro')

In [ ]:
cnf_matrix = metrics.confusion_matrix(targets_argmax,scores_argmax)

In [ ]:
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
#DCF for each class
DCF = (0.75*FNR)+(0.25*FPR)

In [ ]:
min(DCF), np.argmin(DCF)


In [ ]:
targets_argmax = []
scores_argmax = []
for i in targets_con:
    targets_argmax.append(np.argmax(i))

for i in scores_con:
    scores_argmax.append(np.argmax(i))

In [ ]:
# Ground-truth speaker and count

unique, counts = np.unique(targets_argmax, return_counts=True)
dict1 = dict(zip(unique, counts))


In [ ]:
v = [i for i, j in zip(targets_argmax, scores_argmax) if i == j]  #Correctly classified samples based on speakers

In [ ]:
# Correctly classified speaker and count

unique_1, counts_1 = np.unique(v, return_counts=True)
dict2 = dict(zip(unique_1, counts_1))

In [ ]:
for key, value in dict1.items():
    print(key, ' : ', value)  #Ground truth speakers and their segment count

In [ ]:
for key, value in dict2.items():
    print(key, ' : ', value)   #Correctly classified speakers and their segment count

In [ ]:
#Accuracies for each class/speaker.
import operator

accs = []
for i in dict1.keys():
    if i in dict2.keys():
        accs.append(dict2[i]/dict1[i])
    
d = dict(zip(unique_1,accs))
#cd = sorted(d.items(),key=operator.itemgetter(1),reverse=True)
d

In [ ]:
# Mean of the accuracies
mean(accs)

In [ ]:
new_accs = []
for i in accs:
    if i>0.40:
        new_accs.append(i)
        
mean(new_accs),len(new_accs)

In [240]:
def prepare_data_3(example, batch_size, shuffle = False):
    
    #db = Fearless()
    ds = example
    #tr_dict = {}
    #tr_dict['features'] = {} 
    #tr_dict['label_array'] = {}
    
    def prep_features(example):
        
        padded_audio = []
        fbank_data = []
        """ Obtain audio segments from the dataset"""
        """ If segments smaller than 4secs, pad with silence. Else, extract 4secs from larger audio segments """
        audio = pb.io.load_audio(example['audio_path']['observation'],dtype=np.int16)
        if len(audio) < 32000:
            pad = (32000 - len(audio))
            padded = np.concatenate((audio,np.zeros(pad)))
            
        elif len(audio) >= 32000:
            pad = 0
            audio = audio[0:32000]
            padded = audio     
        
        padded_audio.append(padded)
        """ Compute the 64 dimensional filter banks for the 4secs fixed length audio segments"""
    
        fbank = pb.transform.fbank(padded, sample_rate=8000, window_length=400, stft_shift=160,number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97, 
                        window=scipy.signal.windows.hamming)
        fbank_data.append(fbank)
        tens_fbank = torch.FloatTensor(fbank_data)
        #tens_fbank = torch.squeeze(tens_fbank,0)
        #tr_dict['features'] = tens_fbank.to(dev)
        example['features'] = (tens_fbank)
        
        return example
    
    def prep_label(example):
        #label_dict = pb.io.load_json(path = 'labels_argmax.json')
        #label_dict2 = pb.io.load_json(path = 'labels_2.json')
        label_dict3 = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_167.json')
        #print (example)
    #    if example['speaker_id'] in label_dict.keys():
    #    
    #        pos = label_dict[example['speaker_id']]
    #        example['label_array'] = np.array(pos) 
    #        
    #    if example['speaker_id'] in label_dict2.keys():
    #    
    #        pos = label_dict2[example['speaker_id']]
    #        example['label_hot'] = np.array(pos)
    #    return example
    
        if example['speaker_id'] in label_dict3.keys():
        
            pos = label_dict3[example['speaker_id']]
            example['label_array'] = np.array(pos)
        return example
        
    
    def stack(example):
        example['features'] = np.stack(example['features'])
        example['label_array'] = np.stack(example['label_array'])
        #example['label_hot'] = np.stack(example['label_hot'])
        
        return example
    
    example = (example
               .map(prep_features)
               .map(prep_label)
               .batch(batch_size).map(collate_fn)
               .map(stack))
               #.prefetch(buffer_size=8, num_workers=8))
            
            
    return example

In [2]:
from fearless.sid.segmenter import segmenter

In [241]:
m = prepare_data_3(dev_167_lazy,10)
m[0]['features'][0]

array([[[  618.9372  , 20646.082   , 44378.15    , ...,   867.97345 ,
           476.54343 ,   142.91084 ],
        [  367.39133 , 21608.256   , 42563.293   , ...,  1514.2507  ,
           672.3149  ,   123.790375],
        [  534.3272  , 20538.744   , 42540.414   , ...,  1629.2141  ,
           728.6366  ,   123.00544 ],
        ...,
        [  615.92194 , 22682.59    , 46764.773   , ...,  1129.652   ,
           414.83768 ,    92.12422 ],
        [  590.3112  , 22154.275   , 46822.4     , ...,  1825.3707  ,
           802.8954  ,   121.92483 ],
        [ 1238.6941  , 23121.582   , 40701.64    , ...,   901.70166 ,
           906.667   ,   121.90464 ]]], dtype=float32)